In [2]:
from transformers import AutoConfig, AutoTokenizer, AutoModel
from data.acronymDataset import AcronymDataset

In [3]:
model_name = 'microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext'
tokenizer = AutoTokenizer.from_pretrained(model_name)
pre_trained_model = AutoModel.from_pretrained(model_name)
config = AutoConfig.from_pretrained(model_name)

Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Test with the data

In [4]:
file_path = 'data/acronym_data.txt'
dataset = AcronymDataset(file_path=file_path)
dataset = dataset.data

In [5]:
tokens = tokenizer(dataset[0]['source_sentence'], dataset[0]['compare_sentence'], return_tensors='pt')
dataset[16]

{'source_sentence': "She is morbidly obese at 300 pounds, and at that time in _%#MM#%_ of 2001, she defined a dysfunction, irregular bleeding pattern and has a history of a vacuum curettage in _%#MM2001#%_ for an early missed AB at approximately 7 weeks gestation. That was accomplished without difficulty. The patient's blood type is AB positive. She has continued to do well on previous Provera cycling which she has been undergoing.",
 'compare_sentence': "She is morbidly obese at 300 pounds, and at that time in _%#MM#%_ of 2001, she defined a dysfunction, irregular bleeding pattern and has a history of a vacuum curettage in _%#MM2001#%_ for an early missed abortion at approximately 7 weeks gestation. That was accomplished without difficulty. The patient's blood type is AB positive. She has continued to do well on previous Provera cycling which she has been undergoing.",
 'label': 1}

In [6]:
from models.multiHeadModel import MultiHeadModel
from models.heads import ClassificationHead

In [7]:
in_features = config.hidden_size
two_labels_head = ClassificationHead(in_features=in_features, out_features=2)
four_labels_head = ClassificationHead(in_features=in_features, out_features=4)

classifiers = [two_labels_head, four_labels_head]


In [8]:
multi_head_model = MultiHeadModel(pre_trained_model, classifiers)

In [9]:
output = multi_head_model(tokens, 0)
output

tensor([[0.4202, 0.5524]], grad_fn=<SigmoidBackward0>)

In [10]:
output = multi_head_model(tokens, 1)
output

tensor([[0.4424, 0.4114, 0.5453, 0.5039]], grad_fn=<SigmoidBackward0>)

Insert a layer in the middle

In [ ]:
def create_classifier_layer(layer_to_insert):
    till_layer_to_insert = pre_trained_model.encoder.layer[:layer_to_insert]
    after_layer_to_insert = pre_trained_model.encoder.layer[layer_to_insert:]
    model_with_middle_classifier = till_layer_to_insert + [nn.Linear(768, 2)] + after_layer_to_insert

    return model_with_middle_classifier

In [ ]:
model_with_middle_classifier = create_classifier_layer(layer_to_insert=4)
model_with_middle_classifier